In [ ]:
##Allreviews.csv
##https://drive.google.com/file/d/1-7NLpbIG8W6jRXnihqr1XSDXCxZGDPGD/view?usp=sharing
#Check autorisation before reading input file
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
downloaded = drive.CreateFile({'id':"1-7NLpbIG8W6jRXnihqr1XSDXCxZGDPGD"})   # replace the id with id of file you want to access
downloaded.GetContentFile('allreviews.csv')        # replace the file name with your file

In [ ]:
allreviews_df = pd.read_csv("allreviews.csv",sep=",")
allreviews_df.head(100)

,ASIN,ProductFlavor,ProductName,ProductNumReviews,RetrievedTime,ReviewContent,ReviewDomain,ReviewEarly,ReviewHelpful,ReviewId,ReviewLink,ReviewStar,ReviewTime,ReviewVerifiedP,ReviewVine,ReviewersId,ReviewersName,ProcessedText,VaderScore,Effectiveness
0,B0021FEMZG,Chocolate,PlantFusion Complete Plant Based Pea Protein P...,4762,2020-07-22,Used to love this protein shake. A little bit ...,amazon.com,0,584,R24DK56XHP10W8,/gp/customer-reviews/R24DK56XHP10W8,2.0,2018-09-01,1,0,AFJ2JYCF6YCW65ZW7AV2J4YO5MIQ,arrow,use love protein shake little bite sweeter oth...,0.8494,0.0
1,B00I01ZXYE,Flavor_absent,ONE ORGANIC Sushi Nori Premium Roasted Organic...,955,2020-07-22,"One Organic Sushi Nori, 50 sheets = https://ww...",amazon.com,0,92,R2W0O1EYHJDT25,/gp/customer-reviews/R2W0O1EYHJDT25,5.0,2016-10-09,1,0,AHLUVLLSOY2YGWT3H3BWMNB5LREQ,Daniel,one organic sushi nori sheet https www amazon ...,-0.5859,0.0
2,B0021FEMZG,Natural - No Stevia,PlantFusion Complete Plant Based Pea Protein P...,4762,2020-07-22,"Product contains guar gum, AKA carrageenan. Lo...",amazon.com,0,376,R2M5RMJ8V2KT34,/gp/customer-reviews/R2M5RMJ8V2KT34,1.0,2018-09-01,1,0,AHJCHLZ4PJNFAGRZCNJTELPUEJGA,sam smalls,product contain guar gum aka carrageenan look ...,-0.8176,0.0
3,B00I01ZXYE,Flavor_absent,ONE ORGANIC Sushi Nori Premium Roasted Organic...,955,2020-07-22,If I had known this was made in China I would ...,amazon.com,0,37,R16CTB42DC89CS,/gp/customer-reviews/R16CTB42DC89CS,1.0,2019-03-04,1,0,AFRKGOJKOYKAS4HBZNMIROHY6TZQ,josephine,know make china would buy throw mine eat leave...,0.4310,0.0
4,B083SFBKPT,Flavor_absent,"Agar Agar Powder 2oz : Gelatin Substitute, Veg...",587,2020-07-22,This was my first try with this agar-agar and ...,amazon.com,0,152,R2ZH1V4P2CQEN,/gp/customer-reviews/R2ZH1V4P2CQEN,5.0,2017-08-14,1,0,AGXFL2IGLMYHAQFPSA3PSMBD7IHA,Lisa D Ramsey,first try agar agar work great follow instruct...,0.8908,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,B00YQS8ED2,Berry,"Country Farms Super Green Drink, Berry Flavor,...",549,2020-07-22,"Item is not as advertised, does not contain th...",amazon.com,0,18,R2DL03S8BCXWB2,/gp/customer-reviews/R2DL03S8BCXWB2,1.0,2018-01-07,1,0,AH42T7OVXUBWGNTBJNFWT3UPCIFQ,Katie B,item advertise contain mushroom complex list d...,0.0000,0.0
96,B000MGSJ5A,Flavor_absent,"NOW Foods, Agar Pure Powder, Vegetarian Substi...",463,2020-07-22,I got this since this said it's a substitute f...,amazon.com,0,1,R3XMTWN04B6AR,/gp/customer-reviews/R3XMTWN04B6AR,1.0,2020-06-21,1,0,AHYJ3G652PEBNX6DECFOREWKRV3Q,Kiran Yalasangi,get since say substitute gelatin want make man...,0.9687,0.0
97,B002EUZEVY,Flavor_absent,Fresh Whole Leaf Irish Moss - Raw - 16 oz,259,2020-07-22,The moss I received is from Indonesia. Has ver...,amazon.com,0,98,R11FB3181B2K0Q,/gp/customer-reviews/R11FB3181B2K0Q,1.0,2018-04-29,1,0,AHP4WPSPJGVCIV6UVSAIRV5EOOBQ,NATALYA Melnik,moss receive indonesia strong bleach smell sta...,-0.7199,0.0
98,B00YQS8ED2,Chocolate,"Country Farms Super Green Drink, Berry Flavor,...",549,2020-07-22,I’ve been using this product for years. Unfort...,amazon.com,0,2,R3IOE7TMQJ2HG8,/gp/customer-reviews/R3IOE7TMQJ2HG8,3.0,2020-02-17,1,0,AHBULV5MOJDZPNDGJDE6LNED7XAA,DrK,use product years unfortunately certain formul...,0.8374,0.0


In [ ]:
#Importing required library
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

import nltk
from nltk.corpus import stopwords
from nltk.classify import SklearnClassifier

from wordcloud import WordCloud,STOPWORDS
import matplotlib.pyplot as plt
%matplotlib inline

from subprocess import check_output
#Downloading required libraries in nltk
import nltk
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('vader_lexicon')
nltk.download('punkt')

!pip install PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
allreviews_df.count

<bound method DataFrame.count of                                         ASIN  ... Effectiveness
0                                 B0021FEMZG  ...           0.0
1                                 B00I01ZXYE  ...           0.0
2                                 B0021FEMZG  ...           0.0
3                                 B00I01ZXYE  ...           0.0
4                                 B083SFBKPT  ...           0.0
...                                      ...  ...           ...
1538794                           B07N48MTTH  ...           1.0
1538795  B077MYP6H8?reviewerType=all_reviews  ...           0.0
1538796                           B07N48MTTH  ...           1.0
1538797                           B07N48MTTH  ...           0.0
1538798                           B07N48MTTH  ...           1.0

[1538799 rows x 20 columns]>

In [ ]:
from pandas import DataFrame
#Unique ASINs
asin_list=allreviews_df.ASIN.unique()
asin_total=len(asin_list)
print("Total number of unique ASIN",asin_total)

#Unique product names
prod_list=allreviews_df.ProductName.unique()
prod_total=len(prod_list)
print("Total number of unique prod Name",prod_total)
df_prod_name = DataFrame(prod_list,columns=['ProductName'])

Total number of unique ASIN 11087
Total number of unique prod Name 11048


Common Function

In [ ]:
import string
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.tokenize import WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize


def clean_text(text):
    # lower text
    text = text.lower()
    # tokenize text and remove puncutation
    text = [word.strip(string.punctuation) for word in text.split(" ")]
    # remove words that contain numbers
    text = [word for word in text if not any(c.isdigit() for c in word)]
    # remove stop words
    stop = stopwords.words('english')
    text = [x for x in text if x not in stop]
    # remove empty tokens
    text = [t for t in text if len(t) > 0]
    # lemmatize text
    #text = [WordNetLemmatizer().lemmatize(t[0], get_wordnet_pos(t[1])) for t in pos_tags]
    lemmatizer=WordNetLemmatizer()
    text=[lemmatizer.lemmatize(x) for x in text]
    # remove words with only one letter
    text = [t for t in text if len(t) > 1]
    # join all
    text = " ".join(text)
    return(text)


**Ingredients based analysis on allreviews.csv**

In [ ]:
#Function to get the keywords from given input string and string_list
import re

def match(input_string, string_list):
    words = re.findall(r'\w+', input_string)
    return [word for word in words if word in string_list]


In [ ]:
#Function to set indicator for anxiety, stress and sleep
def set_indicator(inputStr):
    if (len(inputStr)==0):
        result = 0
    else:
        result = 1
    return result


Finding ingredients

In [ ]:
#List of ingredients and their various names
lst_l_arginine = ['l-arginine','larginine','2-amino-5(diaminomethylidene amino) pentanoic acid','2-amino-5-guanidinopentanoic acid','(2S)-2-amino-5-{[amino (imino) methyl]amino}pentanoic acid', '(s)-2-amino-5- guanidinopentanoic acid','acide 2-amino-5-guanidinopentanoïque','arginine','arginine ethyl ester','arginine ethyl ester dihydrochloride','arginine ethyl ester hcl','arginine hcl','arginine hydrochloride','di-arginine malate','di-arginine orotate','di-l-arginine-l-malate','dl-arginine','l-arginina','l-arginine ethyl ester dichloride','l-arginine hcl','l-arginine hexanoate','l-arginine hydrochloride','l-arginine ketoisocaproic acid','l-arginine l-pyroglutamate','l-arginine pyroglutamate','l-arginine taurinate','malate de di-arginine','orotate de di-arginine','r-gene 10']

lst_lysine=['lysine','hydrochlorure de l-lysine','l-2,6-diaminohexanoic acid','lisina','l-lysine','l-lysine hcl','l-lysine hydrochloride','l-lysine monohydrochloride','lysine hydrochloride','lysine monohydrochloride','monochlohydrate de l-lysine', 'monochlohydrate de lysine']

lst_lemonbalm =['lemonbalm','lemon balm', 'bálsamo de limón', 'cure-all', 'dropsy plant', 'honey plant', 'melisa', 'melissa', 'melissa officinalis', 'melissae folium', 'mélisse', 'mélisse citronnelle', 'mélisse officinale', 'melissenblatt', 'monarde', 'sweet balm', 'sweet mary', 'toronjil']

lst_ashwagandha=['ashwagandha', 'ajagandha', 'amangura', 'amukkirag', 'asan', 'asana', 'asgand', 'asgandh', 'asgandha', 'ashagandha', 'ashvagandha', 'ashwaganda', 'ashwanga', 'asoda', 'asundha', 'asvagandha',
                 'aswagandha', 'avarada', 'ayurvedic ginseng', 'cerise d hiver', 'clustered wintercherry', 'ghoda asoda', 'ginseng ayurvédique',
                 'ginseng indien', 'hayahvaya', 'indian ginseng', 'kanaje hindi', 'kuthmithi', 'orovale', 'peyette', 'physalis somnifera', 'samm al ferakh',
                 'samm al rerakh', 'sogade-beru', 'strychnos', 'turangi-ghanda', 'vajigandha', 'winter cherry', 'withania', 'withania somnifera', 'ginseng']

lst_gotukola=['gotukola','gotu kola', 'brahma-buti', 'brahma-manduki', 'brahmi', 'centella', 'centella asiatica', 'centella asiática',
              'centella asiatique', 'centellase', 'hydrocotyle', 'hydrocotyle asiatica', 'hydrocotyle asiatique', 'hydrocotyle indien',
              'indischer wassernabel', 'idrocotyle', 'indian pennywort', 'indian water navelwort', 'ji xue cao', 'khulakhudi', 'luei gong gen', 'luo de da',
              'madecassol', 'mandukaparni', 'manduk parani', 'mandukig', 'marsh penny', 'teca', 'ttfca', 'talepetrako', 'thick-leaved pennywort', 'tsubo-kusa',
              'tungchian', 'white rot']

lst_rhodiola = ['rhodiola', 'arctic root', 'extrait de rhodiole', 'golden root', 'hongjingtian', 'kings crown', 'lignum rhodium', 'orpin rose', 'racine d or',
                'racine dorée', 'racine de rhadiola', 'rhodiola rosea', 'rhodiole', 'rhodiole rougeâtre', 'rodia riza', 'rose root', 'rose root extract',
                'rosenroot', 'roseroot', 'rosewort', 'sedum rhodiola', 'sedum rosea', 'siberian golden root', 'siberian rhodiola rosea', 'snowdown rose']

lst_chamomile = ['chamomile','german chamomile','matricaria chamomilla','chamomilla', 'chamomilla recutita',  'matricaria recutita',
                 'matricaria suaveolens']

lst_passionflower=['passionflower', 'passion flower', 'apricot vine', 'burucuya', 'corona de cristo', 'fleischfarbige', 'fleur de la passion',
                   'fleur de passiflore', 'flor de passion', 'granadilla', 'grandilla', 'grenadille', 'madre selva', 'maracuja', 'maypop', 'maypop passion flower',
                   'pasiflora', 'pasionari', 'pasionaria', 'passiflora', 'passiflora incarnata', 'passiflorae herbal', 'passiflore', 'passiflore aubépine',
                   'passiflore officinale', 'passiflore purpurine', 'passiflore rouge', 'passiflorina', 'passion vine', 'passionaria', 'passionblume',
                   'passionflower herb', 'passionsblomma', 'passionsblumenkraut', 'purple passion flower', 'water lemon', 'wild passion flower']

lst_turmeric=['turmeric', 'curcuma', 'curcuma aromatica', 'curcuma domestica', 'curcumae longa', 'curcumae longae rhizoma', 'curcumin', 'curcumine', 'curcuminoid', 'curcumino&iuml;de',
              'curcumino&iuml;des', 'curcuminoids', 'halada', 'haldi', 'haridra', 'indian saffron', 'nisha', 'pian jiang huang', 'racine de curcuma',
              'radix curcumae', 'rajani', 'rhizoma cucurmae longae', 'safran bourbon', 'safran de batallita', 'safran des indes', 'turmeric root', 'yu jin']

lst_bacopa =['andri', 'bacopa', 'bacopa monniera', 'bacopa monnieri', 'herb of grace', 'herpestis herb', 'herpestis monniera', 'hysope d eau', 'indian pennywort',
              'jalanimba', 'jal-brahmi', 'jalnaveri', 'nira-brahmi', 'moniera cuneifolia', 'sambrani chettu', 'thyme-leave gratiola', 'water hyssop']

lst_valarian=['valarian', 'all-heal', 'amantilla', 'baldrian', 'baldrianwurzel', 'belgium valerian', 'common valerian', 'fragrant valerian',
              'garden heliotrope', 'garden valerian', 'grande valériane', 'guérit tout', 'herbe à la femme meurtrie', 'herbe aux chats',
              'herbe aux coupures', 'herbe de notre-dame', 'herbe de saint-georges', 'herbe du loup', 'indian valerian', 'mexican valerian',
              'pacific valerian', 'rhizome de valériane', 'tagar', 'tagar-ganthoda', 'tagara', 'valeriana', 'valeriana angustifolia', 'valeriana edulis',
              'valeriana jatamansii', 'valeriana officinalis', 'valeriana pseudofficinalis', 'valeriana rhizome', 'valeriana sitchensis',
              'valeriana wallichii', 'valerianae radix', 'valeriane', 'valériane', 'valériane à petites feuilles', 'valériane africaine',
              'valériane celtique', 'valériane commune', 'valériane de belgique', 'valériane des collines', 'valériane dioïque', 'valériane du jardin',
              'valériane indienne', 'valériane mexicaine', 'valériane officinale', 'valériane sauvage']

In [ ]:
# clean ProductName data
allreviews_df["ProductName_Clean"] = allreviews_df["ProductName"].apply(lambda x: clean_text(x))

In [ ]:
#Find the ingredient keys
allreviews_df["larginine_key"] = allreviews_df["ProductName_Clean"].apply(lambda x: match(input_string=x, string_list=lst_l_arginine))
allreviews_df["lysine_key"] = allreviews_df["ProductName_Clean"].apply(lambda x: match(input_string=x, string_list=lst_lysine))
allreviews_df["lemonbalm_key"] = allreviews_df["ProductName_Clean"].apply(lambda x: match(input_string=x, string_list=lst_lemonbalm))
allreviews_df["ashwagandha_key"] = allreviews_df["ProductName_Clean"].apply(lambda x: match(input_string=x, string_list=lst_ashwagandha))
allreviews_df["gotukola_key"] = allreviews_df["ProductName_Clean"].apply(lambda x: match(input_string=x, string_list=lst_gotukola))
allreviews_df["rhodiola_key"] = allreviews_df["ProductName_Clean"].apply(lambda x: match(input_string=x, string_list=lst_rhodiola))
allreviews_df["chamomile_key"] = allreviews_df["ProductName_Clean"].apply(lambda x: match(input_string=x, string_list=lst_chamomile))
allreviews_df["passionflower_key"] = allreviews_df["ProductName_Clean"].apply(lambda x: match(input_string=x, string_list=lst_passionflower))
allreviews_df["turmeric_key"] = allreviews_df["ProductName_Clean"].apply(lambda x: match(input_string=x, string_list=lst_turmeric))
allreviews_df["bacopa_key"] = allreviews_df["ProductName_Clean"].apply(lambda x: match(input_string=x, string_list=lst_bacopa))
allreviews_df["valarian_key"] = allreviews_df["ProductName_Clean"].apply(lambda x: match(input_string=x, string_list=lst_valarian))

In [ ]:
#Find the ingredient keys
allreviews_df["larginine_key"] = allreviews_df["ProductName_Clean"].apply(lambda x: match(input_string=x, string_list=lst_l_arginine))
allreviews_df["lysine_key"] = allreviews_df["ProductName_Clean"].apply(lambda x: match(input_string=x, string_list=lst_lysine))
allreviews_df["lemonbalm_key"] = allreviews_df["ProductName_Clean"].apply(lambda x: match(input_string=x, string_list=lst_lemonbalm))
allreviews_df["ashwagandha_key"] = allreviews_df["ProductName_Clean"].apply(lambda x: match(input_string=x, string_list=lst_ashwagandha))
allreviews_df["gotukola_key"] = allreviews_df["ProductName_Clean"].apply(lambda x: match(input_string=x, string_list=lst_gotukola))
allreviews_df["rhodiola_key"] = allreviews_df["ProductName_Clean"].apply(lambda x: match(input_string=x, string_list=lst_rhodiola))
allreviews_df["chamomile_key"] = allreviews_df["ProductName_Clean"].apply(lambda x: match(input_string=x, string_list=lst_chamomile))
allreviews_df["passionflower_key"] = allreviews_df["ProductName_Clean"].apply(lambda x: match(input_string=x, string_list=lst_passionflower))
allreviews_df["turmeric_key"] = allreviews_df["ProductName_Clean"].apply(lambda x: match(input_string=x, string_list=lst_turmeric))
allreviews_df["bacopa_key"] = allreviews_df["ProductName_Clean"].apply(lambda x: match(input_string=x, string_list=lst_bacopa))
allreviews_df["valarian_key"] = allreviews_df["ProductName_Clean"].apply(lambda x: match(input_string=x, string_list=lst_valarian))

In [ ]:
#Get unique ingredient key
allreviews_df['larginine_key'] = allreviews_df['larginine_key'].apply(lambda x: list(pd.unique(x)))
allreviews_df['lysine_key'] = allreviews_df['lysine_key'].apply(lambda x: list(pd.unique(x)))
allreviews_df['lemonbalm_key'] = allreviews_df['lemonbalm_key'].apply(lambda x: list(pd.unique(x)))
allreviews_df['ashwagandha_key'] = allreviews_df['ashwagandha_key'].apply(lambda x: list(pd.unique(x)))
allreviews_df['gotukola_key'] = allreviews_df['gotukola_key'].apply(lambda x: list(pd.unique(x)))

allreviews_df['rhodiola_key'] = allreviews_df['rhodiola_key'].apply(lambda x: list(pd.unique(x)))
allreviews_df['chamomile_key'] = allreviews_df['chamomile_key'].apply(lambda x: list(pd.unique(x)))
allreviews_df['passionflower_key'] = allreviews_df['passionflower_key'].apply(lambda x: list(pd.unique(x)))
allreviews_df['turmeric_key'] = allreviews_df['turmeric_key'].apply(lambda x: list(pd.unique(x)))
allreviews_df['bacopa_key'] = allreviews_df['bacopa_key'].apply(lambda x: list(pd.unique(x)))
allreviews_df['valarian_key'] = allreviews_df['valarian_key'].apply(lambda x: list(pd.unique(x)))

Display the distribution of each ingredient key

In [ ]:
print(allreviews_df["larginine_key"].value_counts())

[]            1538161
[arginine]        638
Name: larginine_key, dtype: int64


In [ ]:
print(allreviews_df["lysine_key"].value_counts())

[]          1538175
[lysine]        624
Name: lysine_key, dtype: int64


In [ ]:
print(allreviews_df["lemonbalm_key"].value_counts())

[]                      1534741
[melissa]                  3978
[melissa, lemonbalm]         74
[toronjil]                    4
[melisa]                      2
Name: lemonbalm_key, dtype: int64


In [ ]:
print(allreviews_df["ashwagandha_key"].value_counts())

[]                                     1453057
[ashwagandha]                            69687
[ginseng]                                 8327
[ashwagandha, ginseng]                    2765
[ashwaganda]                              1634
[ashwagandha, withania]                   1575
[ashwagandha, ashwaganda]                 1388
[ginseng, ashwagandha]                     186
[asana]                                     99
[ashwaganda, ginseng]                       57
[ashwagandha, withania, ginseng]            16
[ashwagandha, withania, ashwaganda]          6
[ashwagandha, ginseng, withania]             2
Name: ashwagandha_key, dtype: int64


In [ ]:
print(allreviews_df["gotukola_key"].value_counts())

[]                        1534992
[brahmi]                     2529
[centella]                    733
[gotukola]                    472
[centella, brahmi]             43
[madecassol, centella]         24
[brahmi, centella]              5
[gotukola, centella]            1
Name: gotukola_key, dtype: int64


In [ ]:
print(allreviews_df["rhodiola_key"].value_counts())

[]            1486082
[rhodiola]      52717
Name: rhodiola_key, dtype: int64


In [ ]:
print(allreviews_df["chamomile_key"].value_counts())

[]                         1465404
[chamomile]                  73385
[chamomile, chamomilla]         10
Name: chamomile_key, dtype: int64


In [ ]:
print(allreviews_df["passionflower_key"].value_counts())

[]                             1536682
[passionflower]                   1575
[passiflora]                       270
[pasiflora]                        259
[passionflower, passiflora]         13
Name: passionflower_key, dtype: int64


In [ ]:
print(allreviews_df["turmeric_key"].value_counts())

[]                                                 1400439
[turmeric]                                           58141
[turmeric, curcumin]                                 35485
[turmeric, curcumin, curcuminoids]                   29147
[curcumin, turmeric]                                  6036
[curcumin]                                            5499
[curcumin, turmeric, curcuminoids]                    1534
[turmeric, curcuma]                                   1300
[turmeric, curcumin, curcuminoid, curcuminoids]        820
[curcumin, curcuminoids, turmeric]                     173
[turmeric, curcumin, curcuma]                           80
[turmeric, curcumin, curcuminoid]                       50
[turmeric, haldi]                                       40
[turmeric, curcumin, curcuminoids, curcuma]             20
[turmeric, curcuminoids]                                11
[curcuma, turmeric]                                     11
[haldi]                                                 

In [ ]:
print(allreviews_df["bacopa_key"].value_counts())

[]          1480567
[bacopa]      58232
Name: bacopa_key, dtype: int64


In [ ]:
print(allreviews_df["valarian_key"].value_counts())

[]             1538096
[valeriana]        356
[valarian]         347
Name: valarian_key, dtype: int64


Display the distribution of ingredients with their combinations

In [ ]:
allreviews_df['ing_list'] = allreviews_df['larginine_key'] + allreviews_df['lysine_key']+ allreviews_df['lemonbalm_key'] + allreviews_df['ashwagandha_key'] + allreviews_df['gotukola_key'] + allreviews_df['rhodiola_key'] + allreviews_df['chamomile_key']+ allreviews_df['passionflower_key'] + allreviews_df['turmeric_key'] + allreviews_df['bacopa_key'] + allreviews_df['valarian_key']


In [ ]:
print(allreviews_df['ing_list'].value_counts())

[]                                                 1183911
[chamomile]                                          63766
[turmeric]                                           53666
[bacopa]                                             36261
[turmeric, curcumin]                                 35391
                                                    ...   
[haldi, turmeric]                                        3
[ashwagandha, ginseng, withania]                         2
[melisa]                                                 2
[gotukola, centella]                                     1
[ashwagandha, rhodiola, curcumin, curcuminoids]          1
Name: ing_list, Length: 95, dtype: int64


In [ ]:
result_df=allreviews_df['ing_list'].value_counts().to_frame()

In [ ]:
result_df.to_csv(r'ingredient_results.csv')

In [ ]:
#Call set_indicator function to set ing_ind
allreviews_df["larginine_ind"] = allreviews_df["larginine_key"].apply(lambda x: set_indicator(x))
allreviews_df["lysine_ind"] = allreviews_df["lysine_key"].apply(lambda x: set_indicator(x))
allreviews_df["lemonbalm_ind"] = allreviews_df["lemonbalm_key"].apply(lambda x: set_indicator(x))
allreviews_df["ashwagandha_ind"] = allreviews_df["ashwagandha_key"].apply(lambda x: set_indicator(x))
allreviews_df["gotukola_ind"] = allreviews_df["gotukola_key"].apply(lambda x: set_indicator(x))

allreviews_df["rhodiola_ind"] = allreviews_df["rhodiola_key"].apply(lambda x: set_indicator(x))
allreviews_df["chamomile_ind"] = allreviews_df["chamomile_key"].apply(lambda x: set_indicator(x))
allreviews_df["passionflower_ind"] = allreviews_df["passionflower_key"].apply(lambda x: set_indicator(x))
allreviews_df["turmeric_ind"] = allreviews_df["turmeric_key"].apply(lambda x: set_indicator(x))
allreviews_df["bacopa_ind"] = allreviews_df["bacopa_key"].apply(lambda x: set_indicator(x))
allreviews_df["valarian_ind"] = allreviews_df["valarian_key"].apply(lambda x: set_indicator(x))

In [ ]:
def set_total_indicator(larginine_ind, lysine_ind, lemonbalm_ind,ashwagandha_ind, gotukola_ind, rhodiola_ind, chamomile_ind, passionflower_ind, turmeric_ind, bacopa_ind,valarian_ind):
    if larginine_ind==1 or lysine_ind==1 or lemonbalm_ind==1 or ashwagandha_ind==1 or gotukola_ind ==1 or rhodiola_ind ==1 or chamomile_ind ==1 or passionflower_ind ==1 or turmeric_ind ==1 or bacopa_ind ==1 or valarian_ind==1:
        result=1
    else:
        result=0
    return result

allreviews_df['ing_found'] = allreviews_df.apply(lambda x: set_total_indicator(x['larginine_ind'], x['lysine_ind'], x['lemonbalm_ind'],x['ashwagandha_ind'], x['gotukola_ind'], x['rhodiola_ind'], x['chamomile_ind'], x['passionflower_ind'], x['turmeric_ind'], x['bacopa_ind'],x['valarian_ind']), axis=1)

In [ ]:
print(allreviews_df['ing_found'].value_counts())

0    1183911
1     354888
Name: ing_found, dtype: int64


Finding ingredients only product name

In [ ]:
# clean ProductName data
df_prod_name["ProductName_Clean"] = df_prod_name["ProductName"].apply(lambda x: clean_text(x))

#Find the ingredient keys
df_prod_name["larginine_key"] = df_prod_name["ProductName_Clean"].apply(lambda x: match(input_string=x, string_list=lst_l_arginine))
df_prod_name["lysine_key"] = df_prod_name["ProductName_Clean"].apply(lambda x: match(input_string=x, string_list=lst_lysine))
df_prod_name["lemonbalm_key"] = df_prod_name["ProductName_Clean"].apply(lambda x: match(input_string=x, string_list=lst_lemonbalm))
df_prod_name["ashwagandha_key"] = df_prod_name["ProductName_Clean"].apply(lambda x: match(input_string=x, string_list=lst_ashwagandha))
df_prod_name["gotukola_key"] = df_prod_name["ProductName_Clean"].apply(lambda x: match(input_string=x, string_list=lst_gotukola))
df_prod_name["rhodiola_key"] = df_prod_name["ProductName_Clean"].apply(lambda x: match(input_string=x, string_list=lst_rhodiola))
df_prod_name["chamomile_key"] = df_prod_name["ProductName_Clean"].apply(lambda x: match(input_string=x, string_list=lst_chamomile))
df_prod_name["passionflower_key"] = df_prod_name["ProductName_Clean"].apply(lambda x: match(input_string=x, string_list=lst_passionflower))
df_prod_name["turmeric_key"] = df_prod_name["ProductName_Clean"].apply(lambda x: match(input_string=x, string_list=lst_turmeric))
df_prod_name["bacopa_key"] = df_prod_name["ProductName_Clean"].apply(lambda x: match(input_string=x, string_list=lst_bacopa))
df_prod_name["valarian_key"] = df_prod_name["ProductName_Clean"].apply(lambda x: match(input_string=x, string_list=lst_valarian))

#Get unique ingredient key
df_prod_name['larginine_key'] = df_prod_name['larginine_key'].apply(lambda x: list(pd.unique(x)))
df_prod_name['lysine_key'] = df_prod_name['lysine_key'].apply(lambda x: list(pd.unique(x)))
df_prod_name['lemonbalm_key'] = df_prod_name['lemonbalm_key'].apply(lambda x: list(pd.unique(x)))
df_prod_name['ashwagandha_key'] = df_prod_name['ashwagandha_key'].apply(lambda x: list(pd.unique(x)))
df_prod_name['gotukola_key'] = df_prod_name['gotukola_key'].apply(lambda x: list(pd.unique(x)))

df_prod_name['rhodiola_key'] = df_prod_name['rhodiola_key'].apply(lambda x: list(pd.unique(x)))
df_prod_name['chamomile_key'] = df_prod_name['chamomile_key'].apply(lambda x: list(pd.unique(x)))
df_prod_name['passionflower_key'] = df_prod_name['passionflower_key'].apply(lambda x: list(pd.unique(x)))
df_prod_name['turmeric_key'] = df_prod_name['turmeric_key'].apply(lambda x: list(pd.unique(x)))
df_prod_name['bacopa_key'] = df_prod_name['bacopa_key'].apply(lambda x: list(pd.unique(x)))
df_prod_name['valarian_key'] = df_prod_name['valarian_key'].apply(lambda x: list(pd.unique(x)))

In [ ]:
df_prod_name['ing_list'] = df_prod_name['larginine_key'] + df_prod_name['lysine_key']+ df_prod_name['lemonbalm_key'] + df_prod_name['ashwagandha_key'] + df_prod_name['gotukola_key'] + df_prod_name['rhodiola_key'] + df_prod_name['chamomile_key']+ df_prod_name['passionflower_key'] + df_prod_name['turmeric_key'] + df_prod_name['bacopa_key'] + df_prod_name['valarian_key']

print(df_prod_name['ing_list'].value_counts())

result_df1=df_prod_name['ing_list'].value_counts().to_frame()

[]                                             8856
[turmeric]                                      453
[chamomile]                                     451
[rhodiola]                                      173
[ashwagandha]                                   161
                                               ... 
[haldi, turmeric]                                 1
[haldi]                                           1
[turmeric, curcumin, curcuma, haldi]              1
[turmeric, haldi]                                 1
[turmeric, curcumin, curcuminoids, curcuma]       1
Name: ing_list, Length: 95, dtype: int64


In [ ]:
result_df.rename(columns = {'ing_list':'Review_Count'}, inplace = True)
result_df1.rename(columns = {'ing_list':'Product_Count'}, inplace = True)
result_df.to_csv(r'ingredient_review_count')
result_df1.to_csv(r'ingredient_product_count')

In [ ]:
result_df


,Review_Count
[],1183911
[chamomile],63766
[turmeric],53666
[bacopa],36261
"[turmeric, curcumin]",35391
...,...
"[haldi, turmeric]",3
"[ashwagandha, ginseng, withania]",2
[melisa],2
"[gotukola, centella]",1


In [ ]:
result_df1


,Product_Count
[],8856
[turmeric],453
[chamomile],451
[rhodiola],173
[ashwagandha],161
...,...
"[haldi, turmeric]",1
[haldi],1
"[turmeric, curcumin, curcuma, haldi]",1
"[turmeric, haldi]",1
